In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

In [2]:
import requests
import time
import json

BASE_URL = "https://pncp.gov.br/api/consulta/v1/"

# Define os parametros da api
params = {
    "dataInicial": "20230101",  
    "dataFinal": "20231231",    
    "codigoModalidadeContratacao": 8,  # 8 corresponds to Dispensa de Licitação
    "pagina": 75
}

# Iniciando a lista para armazenar os dados
todos_os_dados = []

# Função para consumir a API
def fetch_data(params, retries=3):
    for attempt in range(retries):
        try:
            print(f"Requesting page {params['pagina']} with parameters: {params}")
            response = requests.get(BASE_URL + "contratacoes/publicacao", params=params)
            
            if response.status_code == 200:
                return response.json()  # Retorna JSON se sucesso
            else:
                print(f"Error: {response.status_code}. Response: {response.text}. Attempt {attempt+1}/{retries}")
        except requests.RequestException as e:
            print(f"Request error: {e}. Attempt {attempt+1}/{retries}")
        time.sleep(1)  # Aguarda antes de tentar novamente
    return None  # Retorna None se todas as tentivas falharem

# Function to save partial results to a file
def save_partial_results(data, filename="partial_results.json"):
    with open(filename, "a") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.write("\n")

# Primeira chamada da API
result = fetch_data(params)

if result:
    todos_os_dados.extend(result.get('data', []))  # Coletando a primeira página dos dados
    total_paginas = result.get('totalPaginas', 1)  # Obtendo o total de paginas
    
    # Salva a primeira págia de dados
    save_partial_results(result.get('data', []))

    # Loop enquanto existir páginas
    for pagina in range(2, total_paginas + 1):
        params["pagina"] = pagina  # Atualiza o número de página
        
        result = fetch_data(params)
        if result:
            todos_os_dados.extend(result.get('data', []))
            
            # Salva cada página de forma incremental.
            save_partial_results(result.get('data', []))
        else:
            print(f"Failed to retrieve data for page {pagina}")

    print(f"Total records collected: {len(todos_os_dados)}")
else:
    print("Failed to retrieve any data.")


Requesting page 75 with parameters: {'dataInicial': '20230101', 'dataFinal': '20231231', 'codigoModalidadeContratacao': 8, 'pagina': 75}
Requesting page 2 with parameters: {'dataInicial': '20230101', 'dataFinal': '20231231', 'codigoModalidadeContratacao': 8, 'pagina': 2}


In [38]:
import pandas as pd
import json
import re

# Lista para armazenar os dados
dados = []

# Carregue o arquivo JSON
with open('partial_results.json', 'r', encoding='Windows-1252') as f:
    # Lê todo o conteúdo
    conteudo = f.read()
    # Substituir vírgulas duplicadas usando regex
    conteudo = re.sub(r'},\s*,', '},', conteudo)
    # Divide por colchetes (assumindo que não há colchetes aninhados)
    objetos = conteudo.split('][')
    for objeto in objetos:
        # Corrige a formatação para cada objeto
        objeto = objeto.strip().strip('[]')
        if objeto:  # Se não estiver vazio
            try:
                dados.extend(json.loads(f"[{objeto}]"))  # Adiciona colchetes para formar uma lista
            except json.JSONDecodeError as e:
                print(f"Erro ao decodificar JSON: {e}")

# Converta a lista de objetos em um DataFrame
df = pd.DataFrame(dados)

# Exiba o DataFrame
df

,valorTotalHomologado,modalidadeId,srp,anoCompra,sequencialCompra,orgaoSubRogado,orgaoEntidade,amparoLegal,dataAberturaProposta,dataEncerramentoProposta,...,linkProcessoEletronico,modoDisputaId,valorTotalEstimado,modalidadeNome,modoDisputaNome,tipoInstrumentoConvocatorioNome,tipoInstrumentoConvocatorioCodigo,situacaoCompraId,situacaoCompraNome,usuarioNome
0,950893.42,8,False,2023,1,None,"{'cnpj': '33654831000136', 'razaoSocial': 'CON...","{'codigo': 24, 'descricao': 'Dispensa de Licit...",None,None,...,None,5,950893.42,Dispensa,Nï¿½o se aplica,Ato que autoriza a Contrataï¿½ï¿½o Direta,3,1,Divulgada no PNCP,Compras.gov.br
1,11800.92,8,False,2023,2,None,"{'cnpj': '33654831000136', 'razaoSocial': 'CON...","{'codigo': 24, 'descricao': 'Dispensa de Licit...",None,None,...,None,5,11800.92,Dispensa,Nï¿½o se aplica,Ato que autoriza a Contrataï¿½ï¿½o Direta,3,1,Divulgada no PNCP,Compras.gov.br
2,59065.49,8,False,2023,3,None,"{'cnpj': '33654831000136', 'razaoSocial': 'CON...","{'codigo': 24, 'descricao': 'Dispensa de Licit...",None,None,...,None,5,59065.49,Dispensa,Nï¿½o se aplica,Ato que autoriza a Contrataï¿½ï¿½o Direta,3,1,Divulgada no PNCP,Compras.gov.br
3,1920.00,8,False,2023,1,None,"{'cnpj': '15126437000143', 'razaoSocial': 'EMP...","{'codigo': 19, 'descricao': 'Dispensa de Licit...",2023-01-02T09:08:48,2023-01-05T07:59:59,...,None,4,0.00,Dispensa,Dispensa Com Disputa,Aviso de Contrataï¿½ï¿½o Direta,2,1,Divulgada no PNCP,Compras.gov.br
4,4130.75,8,False,2023,4,None,"{'cnpj': '33654831000136', 'razaoSocial': 'CON...","{'codigo': 24, 'descricao': 'Dispensa de Licit...",None,None,...,None,5,4130.75,Dispensa,Nï¿½o se aplica,Ato que autoriza a Contrataï¿½ï¿½o Direta,3,1,Divulgada no PNCP,Compras.gov.br
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,120.00,8,False,2023,2,None,{'razaoSocial': 'FUNDO MUNICIPAL DE SAUDE DO M...,"{'codigo': 18, 'descricao': 'Dispensa de Licit...",None,None,...,None,5,120.00,Dispensa,Nï¿½o se aplica,Ato que autoriza a Contrataï¿½ï¿½o Direta,3,1,Divulgada no PNCP,Megasoft Informï¿½tica
2963,1000.00,8,False,2023,3,None,{'razaoSocial': 'FUNDO MUNICIPAL DE SAUDE DO M...,"{'codigo': 19, 'descricao': 'Dispensa de Licit...",None,None,...,None,5,1000.00,Dispensa,Nï¿½o se aplica,Ato que autoriza a Contrataï¿½ï¿½o Direta,3,1,Divulgada no PNCP,Megasoft Informï¿½tica
2964,10000.00,8,False,2023,4,None,{'razaoSocial': 'FUNDO MUNICIPAL DE SAUDE DO M...,"{'codigo': 19, 'descricao': 'Dispensa de Licit...",None,None,...,None,5,10000.00,Dispensa,Nï¿½o se aplica,Ato que autoriza a Contrataï¿½ï¿½o Direta,3,1,Divulgada no PNCP,Megasoft Informï¿½tica
2965,1000.00,8,False,2023,6,None,{'razaoSocial': 'FUNDO MUNICIPAL DE SAUDE DO M...,"{'codigo': 19, 'descricao': 'Dispensa de Licit...",None,None,...,None,5,1000.00,Dispensa,Nï¿½o se aplica,Ato que autoriza a Contrataï¿½ï¿½o Direta,3,1,Divulgada no PNCP,Megasoft Informï¿½tica


In [39]:
df.to_csv("dados_coletados_PNCP_ate_pagina_74.csv")

In [40]:
df_normalize = pd.json_normalize(dados)
df.to_csv("dados_coletados_PNCP_ate_pagina_74_normalize.csv")

In [42]:
df_normalize

,valorTotalHomologado,modalidadeId,srp,anoCompra,sequencialCompra,orgaoSubRogado,dataAberturaProposta,dataEncerramentoProposta,informacaoComplementar,processo,...,orgaoSubRogado.razaoSocial,orgaoSubRogado.poderId,orgaoSubRogado.esferaId,orgaoSubRogado.cnpj,unidadeSubRogada.ufNome,unidadeSubRogada.codigoUnidade,unidadeSubRogada.nomeUnidade,unidadeSubRogada.ufSigla,unidadeSubRogada.municipioNome,unidadeSubRogada.codigoIbge
0,950893.42,8,False,2023,1,NaN,None,None,Melhor custo beneficio,01300008898202266,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11800.92,8,False,2023,2,NaN,None,None,Os equipamentos de mediï¿½ï¿½o da radiaï¿½ï¿½o...,01300009365202274,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59065.49,8,False,2023,3,NaN,None,None,Com a experiï¿½ncia em pesquisa de campo hï¿½ ...,01300012908202268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1920.00,8,False,2023,1,NaN,2023-01-02T09:08:48,2023-01-05T07:59:59,Atendimento ao DESPACHO DECISï¿½RIO - SEI Nï¿½...,23818008936202111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4130.75,8,False,2023,4,NaN,None,None,Melhor empresa do mercado.,01300012215202275,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,120.00,8,False,2023,2,NaN,None,None,None,8272/2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2963,1000.00,8,False,2023,3,NaN,None,None,None,38/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2964,10000.00,8,False,2023,4,NaN,None,None,None,37/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2965,1000.00,8,False,2023,6,NaN,None,None,None,33/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
